### In this notebook, we will see the use of fuzzy string matching.
Sometimes we see that we are trying to refer to the same thing in different ways because it is written differently or it is misspelled or we have typos. This problem often occurs in databases.So, lets see how we can fuzzy match strings.

When we usually compare strings in python like the below:

In [1]:
str1 = "fuzzy wuzzy"
str2 = "fuzzy wuzzy"
Result = str1 == str2
print(Result)

True


Let's see what happens if any of the string changes

In [2]:
str1 = "Fuzzy wuzzy"
str2 = "fuzzy wuzzy"
Result = str1 == str2
print(Result)

False


We got False even if both the strings mena the same company. So, if we convert both strings to lower case, we will solve the problem

In [3]:
str1 = "Fuzzy wuzzy"
str2 = "fuzzy wuzzy"
Result = str1.lower() == str2.lower()
print(Result)

True


Problem comes when characters are missing from the strings. See below:

In [4]:
str1 = "Fuzzy wuzzy"
str2 = "fuzzy wuzzy"
Result = str1.lower() == str2.lower()
print(Result)

True


These type of problems often occur in databases and we therefore need strong tools to compare strings. One such tools is 'Levenshtein distance'.

### Levenshtein Distance
It is a metric used to measure distance between two strings meaning how apart are two sequence of words. It measures the minimum number of edits that should be done to change a one-word sequence to the other.The edits can be insertion, deletion or substitutions.

In [5]:
import numpy as np
def levenshtein_ratio_and_distance(s, t, ratio_calc = False):
    """ levenshtein_ratio_and_distance:
        Calculates levenshtein distance between two strings.
        If ratio_calc = True, the function computes the
        levenshtein distance ratio of similarity between two strings
        For all i and j, distance[i,j] will contain the Levenshtein
        distance between the first i characters of s and the
        first j characters of t
    """
    # Initialize matrix of zeros
    rows = len(s)+1
    cols = len(t)+1
    distance = np.zeros((rows,cols),dtype = int)

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        return "The strings are {} edits away".format(distance[row][col])

In [6]:
str1 = "Fuzzy wuzzy."
str2 = "fuzzy wuzzy"
Distance = levenshtein_ratio_and_distance(str1,str2)
print(Distance)
Ratio = levenshtein_ratio_and_distance(str1,str2,ratio_calc = True)
print(Ratio)

The strings are 2 edits away
0.8695652173913043


If you do very simple string preprocessing before calculating distance we will see that the output changes.

In [7]:
str1 = "fuzzy wuzzy."
str2 = "fuzzy wuzzy"
Distance = levenshtein_ratio_and_distance(str1.lower(),str2.lower())
print(Distance)
Ratio = levenshtein_ratio_and_distance(str1,str2,ratio_calc = True)
print(Ratio)

The strings are 1 edits away
0.9565217391304348


We see that, the distance has been reduced by 1 simply because the strings have been turned to lower case before comparing and hence, the similarity ratio reaches almost 95%. This emphasizes the relevance of string preprocessing before performing calculations. If we were, say, choosing if a string is similar to another one based on a similarity threshold of 90%, then "Apple Inc." and "apple Inc" without preprocessing would be marked as not similar.
Lets try to use the levenshtein package in python

In [10]:
pip install Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 12.4 MB/s eta 0:00:00


In [11]:
import Levenshtein as lev
str1 = "fuzzy wuzzy."
str2 = "fuzzy Wuzzy"
Distance = lev.distance(str1.lower(),str2.lower()),
print(Distance)
Ratio = lev.ratio(str1.lower(),str2.lower())
print(Ratio)

(1,)
0.9565217391304348


### Lets explore FuzzyWuzzy package
This package in python is very useful when the levenshtein distance ratio of similarity between two strings falls short. Our examples were very simple where we had difference in cases (lower/upper) and periods. But what happens when something is spelled out of order?
What happens when has a lot of variation in spelling but it still refers to the same thing?
FuzzyWuzzy package is to the rescue because it has functions that allow our fuzzy matching scripts to handle these sorts of cases.

FuzzyWuzzy has, just like the Levenshtein package, a ratio function that computes the standard Levenshtein distance similarity ratio between two sequences.

In [13]:
pip install fuzzywuzzy

In [14]:
from fuzzywuzzy import fuzz
str1 = "fuzzy Wuzzy."
str2 = "fuzzy wuzzy"
Ratio = fuzz.ratio(str1.lower(),str2.lower())
print(Ratio)

96


This is more like we saw in the above examples. FuzzyWuzzy has more powerful functions that allow us to deal with more complex situations such as substring matching. See below:

In [15]:
str1 = "Seattle Park Riders"
str2 = "Riders"
Ratio = fuzz.ratio(str1.lower(),str2.lower())
Partial_Ratio = fuzz.partial_ratio(str1.lower(),str2.lower())
print(Ratio)
print(Partial_Ratio)

48
100


Using the fuzz.partial_ratio() function, we are able to detect that both strings are referring to the Riders. Thus, we see 100% similarity. This works using an "optimal partial" logic. So, what happens is if the short string has length k and the longer string has the length m, then the algorithm seeks the score of the best matching length-k substring.

This approach is not foolproof. What happens when the strings comparison is the same, but they are in a different order? Fuzzywuzzy is useful here:

In [16]:
str1 = "India vs New Zealand World Cup"
str2 = "New Zealand World Cup vs India"
Ratio = fuzz.ratio(str1.lower(),str2.lower())
Partial_Ratio = fuzz.partial_ratio(str1.lower(),str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(str1,str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)

70
70
100


In the function token_sort_ratio(), the string tokens get sorted alphabetically and then joined together. After that, a simple fuzz.ratio() is applied to obtain the similarity percentage. This allows us to judge if the strings are actually the same.

Now, the question is what happens if these two strings are of widely differing lengths? Thats where fuzz.token_set_ratio() comes in. See below example:

In [17]:
str1 = "The supreme court case of Facebook vs The United States"
str2 = "Facebook vs United States"
Ratio = fuzz.ratio(str1.lower(),str2.lower())
Partial_Ratio = fuzz.partial_ratio(str1.lower(),str2.lower())
Token_Sort_Ratio = fuzz.token_sort_ratio(str1,str2)
Token_Set_Ratio = fuzz.token_set_ratio(str1,str2)
print(Ratio)
print(Partial_Ratio)
print(Token_Sort_Ratio)
print(Token_Set_Ratio)

62
84
62
100


So, here in the above scenario, instead of just tokenizing the strings, sorting and then joining the tokens back together, token_set_ratio performs a set operation that takes out the common tokens (the intersection) and then makes fuzz.ratio() pairwise comparisons between the following new strings:

s1 = Sorted_tokens_in_intersection
s2 = Sorted_tokens_in_intersection + sorted_rest_of_str1_tokens
s3 = Sorted_tokens_in_intersection + sorted_rest_of_str2_tokens

Finally, the fuzzywuzzy package has a module called process that allows you to calculate the string with the highest similarity out of a vector of strings.



In [18]:
from fuzzywuzzy import process
str2Match = "fuzzy wuzzy"
strOptions = ["fuzzy Wuzzy.","Fuzzy park","Fuzzy incorporated","fuzz"]
Ratios = process.extract(str2Match,strOptions)
print(Ratios)
# You can also select the string with the highest matching percentage
highest = process.extractOne(str2Match,strOptions)
print(highest)

[('fuzzy Wuzzy.', 100), ('fuzz', 90), ('Fuzzy incorporated', 86), ('Fuzzy park', 64)]
('fuzzy Wuzzy.', 100)
